In [30]:
import numpy as np

In [31]:
uu = np.load('../data/DUCT_3500/DUCT_3500_uu.npy')
uv = np.load('../data/DUCT_3500/DUCT_3500_uv.npy')
uw = np.load('../data/DUCT_3500/DUCT_3500_uw.npy')
vv = np.load('../data/DUCT_3500/DUCT_3500_vv.npy')
vw = np.load('../data/DUCT_3500/DUCT_3500_vw.npy')
ww = np.load('../data/DUCT_3500/DUCT_3500_ww.npy')

epsi = np.load('../data/DUCT_3500/kepsilon_DUCT_3500_epsilon.npy')
k = np.load('../data/DUCT_3500/kepsilon_DUCT_3500_k.npy')
gradU = np.load('../data/DUCT_3500/kepsilon_DUCT_3500_gradU.npy')

#tensor = np.load('../data/kepsilon_DUCT_1100_Tensors.npy')

In [32]:
ww.shape

(9216,)

In [33]:
k.shape

(9216,)

In [34]:
gradU.shape

(9216, 3, 3)

In [35]:
grad_u_00 = []
grad_u_01 = []
grad_u_02 = []
grad_u_10 = []
grad_u_11 = []
grad_u_12 = []
grad_u_20 = []
grad_u_21 = []
grad_u_22 = [] 
for i in np.arange(gradU.shape[0]):
    grad_u_00.append(gradU[i,0,0])
    grad_u_01.append(gradU[i,0,1])
    grad_u_02.append(gradU[i,0,2])
    grad_u_10.append(gradU[i,1,0])
    grad_u_11.append(gradU[i,1,1])
    grad_u_12.append(gradU[i,1,2])
    grad_u_20.append(gradU[i,2,0])
    grad_u_21.append(gradU[i,2,1])
    grad_u_22.append(gradU[i,2,2]) 
grad_u_00 = np.array(grad_u_00)
grad_u_01 = np.array(grad_u_01)
grad_u_02 = np.array(grad_u_02)
grad_u_10 = np.array(grad_u_10)
grad_u_11 = np.array(grad_u_11)
grad_u_12 = np.array(grad_u_12)
grad_u_20 = np.array(grad_u_20)
grad_u_21 = np.array(grad_u_21)
grad_u_22 = np.array(grad_u_22)   

In [36]:
grad_u_00.shape

(9216,)

In [37]:
# If this cell does not work, please change 'r+' by 'a', and delete the last file before 
# running this cell every time 
f = open ("../data/DUCT_3500/data_reshaped.txt", "r+")
f.write('tke, epsilon, grad_u_00, grad_u_01, grad_u_02, grad_u_10, grad_u_11, grad_u_12, grad_u_20, grad_u_21, grad_u_22,uu_00, uu_01, uu_02, uu_10, uu_11, uu_12, uu_20, uu_21, uu_22, uu,uv,uw,vu,vv,vw,wu,wv,ww \n')
for i in np.arange(len(k)):
    f.write(
            f"{k[i]:.3E}"+'   '+f"{epsi[i]:.3E}"+'   '+
            f"{grad_u_00[i]:.3E}"+'   '+f"{grad_u_01[i]:.3E}"+'   '+f"{grad_u_02[i]:.3E}"+'   '+
            f"{grad_u_10[i]:.3E}"+'   '+f"{grad_u_11[i]:.3E}"+'   '+f"{grad_u_12[i]:.3E}"+'   '+
            f"{grad_u_20[i]:.3E}"+'   '+f"{grad_u_21[i]:.3E}"+'   '+f"{grad_u_22[i]:.3E}"+'   '+
            f"{uu[i]:.3E}"+'   '+f"{uv[i]:.3E}"+'   '+f"{uw[i]:.3E}"+'   '+
            f"{uv[i]:.3E}"+'   '+f"{vv[i]:.3E}"+'   '+f"{vw[i]:.3E}"+'   '+
            f"{uw[i]:.3E}"+'   '+f"{vw[i]:.3E}"+'   '+f"{ww[i]:.3E}"+'   '+'\n'
           )
f.close()

In [38]:
data = np.loadtxt('../data/DUCT_3500/data_reshaped.txt', skiprows=1)

In [39]:
data.shape

(9216, 20)

In [40]:
tke = data[:, 0]
eps = data[:, 1]
grad_u_flat = data[:, 2:11]
stresses_flat = data[:, 11:]

# Reshape grad_u and stresses to num_points X 3 X 3 arrays
num_points = data.shape[0]
grad_u = np.zeros((num_points, 3, 3))
stresses = np.zeros((num_points, 3, 3))
for i in range(3):
    for j in range(3):
        grad_u[:, i, j] = grad_u_flat[:, i*3+j]
        stresses[:, i, j] = stresses_flat[:, i*3+j]

In [41]:
cap = 7
eps = np.maximum(eps, 1e-8)
tke_eps = tke / eps

Sij = np.zeros((num_points, 3, 3))
Rij = np.zeros((num_points, 3, 3))

In [42]:
for i in range(num_points):
    Sij[i, :, :] = 0.5 * (grad_u[i, :, :] + np.transpose(grad_u[i, :, :]))
    Rij[i, :, :] = 0.5 * (grad_u[i, :, :] - np.transpose(grad_u[i, :, :]))

In [43]:
Sij[0]

array([[ 0.00000000e+00, -9.65499999e-04,  1.66000000e+01],
       [-9.65499999e-04, -2.50700000e-12, -2.27473915e-10],
       [ 1.66000000e+01, -2.27473915e-10, -9.35400000e-12]])

In [44]:
Rij[0]

array([[ 0.00000000e+00, -9.65500001e-04,  1.66000000e+01],
       [ 9.65500001e-04,  0.00000000e+00, -2.27526085e-10],
       [-1.66000000e+01,  2.27526085e-10,  0.00000000e+00]])

In [45]:
num_points = Sij.shape[0]
num_invariants = 5
invariants = np.zeros((num_points, num_invariants))
for i in range(num_points):
    invariants[i, 0] = np.trace(np.dot(Sij[i, :, :], Sij[i, :, :]))
    invariants[i, 1] = np.trace(np.dot(Rij[i, :, :], Rij[i, :, :]))
    invariants[i, 2] = np.trace(np.dot(Sij[i, :, :], np.dot(Sij[i, :, :], Sij[i, :, :])))
    invariants[i, 3] = np.trace(np.dot(Rij[i, :, :], np.dot(Rij[i, :, :], Sij[i, :, :])))
#     invariants[i, 3] = np.trace(np.dot(Sij[i, :, :], np.dot(Rij[i, :, :], Rij[i, :, :])))
    invariants[i, 4] = np.trace(np.dot(np.dot(Rij[i, :, :], Rij[i, :, :]), np.dot(Sij[i, :, :], Sij[i, :, :])))

In [46]:
num_tensor_basis = 10
T = np.zeros((num_points, num_tensor_basis, 3, 3))
for i in range(num_points):
    sij = Sij[i, :, :]
    rij = Rij[i, :, :]
    T[i, 0, :, :] = sij
    T[i, 1, :, :] = np.dot(sij, rij) - np.dot(rij, sij)
    T[i, 2, :, :] = np.dot(sij, sij) - 1./3.*np.eye(3)*np.trace(np.dot(sij, sij))
    T[i, 3, :, :] = np.dot(rij, rij) - 1./3.*np.eye(3)*np.trace(np.dot(rij, rij))
    T[i, 4, :, :] = np.dot(rij, np.dot(sij, sij)) - np.dot(np.dot(sij, sij), rij)
    T[i, 5, :, :] = np.dot(rij, np.dot(rij, sij)) \
                    + np.dot(sij, np.dot(rij, rij)) \
                    - 2./3.*np.eye(3)*np.trace(np.dot(sij, np.dot(rij, rij)))
    T[i, 6, :, :] = np.dot(np.dot(rij, sij), np.dot(rij, rij)) - np.dot(np.dot(rij, rij), np.dot(sij, rij))
    T[i, 7, :, :] = np.dot(np.dot(sij, rij), np.dot(sij, sij)) - np.dot(np.dot(sij, sij), np.dot(rij, sij))
    T[i, 8, :, :] = np.dot(np.dot(rij, rij), np.dot(sij, sij)) \
                    + np.dot(np.dot(sij, sij), np.dot(rij, rij)) \
                    - 2./3.*np.eye(3)*np.trace(np.dot(np.dot(sij, sij), np.dot(rij, rij)))
    T[i, 9, :, :] = np.dot(np.dot(rij, np.dot(sij, sij)), np.dot(rij, rij)) \
                    - np.dot(np.dot(rij, np.dot(rij, sij)), np.dot(sij, rij))
    # Enforce zero trace for anisotropy
    for j in range(num_tensor_basis):
        T[i, j, :, :] = T[i, j, :, :] - 1./3.*np.eye(3)*np.trace(T[i, j, :, :])

In [47]:
np.trace(T[0,0])

0.0